In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA

In [5]:
train_df_original = pd.read_csv('TRAIN_FINAL.csv')
test_df = pd.read_csv('Test_v1.csv')

In [6]:
positive_dataset = train_df_original[train_df_original['TARGET']==0]

In [12]:
len(negative_dataset)

1763

In [27]:
positive_dataset = train_df_original[train_df_original['TARGET']==0]

negative_dataset = train_df_original[train_df_original['TARGET']==1]

sample_positive5 = positive_dataset[:17630]
train_df5 = pd.concat([sample_positive5,negative_dataset])
train_df5 = train_df5.sample(frac = 1,random_state=42)

sample_positive10 =positive_dataset[17630:19393]
train_df10 = pd.concat([sample_positive10,negative_dataset])
train_df10 = train_df10.sample(frac = 1,random_state=42)

sample_positive15 = positive_dataset[35260:52890]
train_df15 = pd.concat([sample_positive15,negative_dataset])
train_df15 = train_df15.sample(frac = 1,random_state=42)

In [28]:
X_5 = train_df5.drop(['TARGET'],axis=1)

y_5 = train_df5['TARGET']

X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split( X_5, y_5, test_size=0.33, random_state=42,stratify = y_5)


X_10 = train_df10.drop(['TARGET'],axis=1)

y_10 = train_df10['TARGET']

X_train_10, X_test_10, y_train_10, y_test_10 = train_test_split( X_10, y_10, test_size=0.33, random_state=43,stratify = y_10)

X_15 = train_df15.drop(['TARGET'],axis=1)

y_15 = train_df15['TARGET']

X_train_15, X_test_15, y_train_15, y_test_15 = train_test_split( X_15, y_15, test_size=0.33, random_state=44,stratify = y_15)

In [15]:
cat_15 = XGBClassifier(n_estimators=40,max_depth=3,scale_pos_weight=15)

cat_15.fit(X_train_15, y_train_15)

pred = cat_15.predict(X_test_15)
print(roc_auc_score(y_test_15,pred))

0.698409871915


In [38]:
#10

cat_10 = XGBClassifier(n_estimators=150,max_depth=5)

cat_10.fit(X_train_10, y_train_10)

pred = cat_10.predict(X_test_10)
print(roc_auc_score(y_test_10,pred))

0.759450171821


In [39]:
#5
from xgboost import XGBClassifier

cat_5 = XGBClassifier(n_estimators=100,max_depth=2,scale_pos_weight=10)

cat_5.fit(X_train_5, y_train_5)

pred = cat_5.predict(X_test_5)
print(roc_auc_score(y_test_5,pred))


#10

cat_10 = XGBClassifier(n_estimators=150,max_depth=5)

cat_10.fit(X_train_10, y_train_10)

pred = cat_10.predict(X_test_10)
print(roc_auc_score(y_test_10,pred))


#15

cat_15 = XGBClassifier(n_estimators=100,max_depth=3,scale_pos_weight=10)

cat_15.fit(X_train_15, y_train_15)

pred = cat_15.predict(X_test_15)
print(roc_auc_score(y_test_15,pred))


0.767484545533
0.759450171821
0.744385352461


In [42]:
#stacking test

# stacking dataset

cat_5_out =  cat_5.predict_proba(X_test_15)[:,1]
cat_10_out = cat_10.predict_proba(X_test_15)[:,1]
cat_15_out = cat_15.predict_proba(X_test_15)[:,1]
pred = ((cat_5_out + cat_10_out + cat_15_out)/3 > 0.53).astype('int')
print(roc_auc_score(y_test_15,pred))

0.783020600375


In [41]:
import numpy as np
for i in np.arange(0.4,0.6,0.005):
    pred = ((cat_5_out + cat_10_out + cat_15_out)/3 > i).astype('int')
    print(i,roc_auc_score(y_test_15,pred))

0.4 0.770909471081
0.405 0.771461299296
0.41 0.770977654139
0.415 0.771705849197
0.42 0.773844979
0.425 0.774419534901
0.43 0.77453317333
0.435 0.773190421644
0.44 0.775329551446
0.445 0.776234113348
0.45 0.775949562719
0.455 0.777536864307
0.46 0.777758686522
0.465 0.779698721795
0.47 0.779237804324
0.475 0.781001472754
0.48 0.782082401498
0.485 0.783163330242
0.49 0.783032418771
0.495 0.7829015073
0.5 0.780016909398
0.505 0.780745104456
0.51 0.781473299515
0.515 0.7808360152
0.52 0.781740577101
0.525 0.781080565101
0.53 0.783020600375
0.535 0.779805996473
0.54 0.77954417353
0.545 0.776858670158
0.55 0.776221385843
0.555 0.777501409117
0.56 0.776886852488
0.565 0.775743195331
0.57 0.773740431644
0.575 0.773632247859
0.58 0.774912271132
0.585 0.774781359661
0.59 0.775708649248
0.595 0.774564992091


In [43]:
ind = test_df['index']
test = test_df.drop(['index'],axis=1)

In [44]:
cat_5_out =  cat_5.predict_proba(test)[:,1]
cat_10_out = cat_10.predict_proba(test)[:,1]
cat_15_out = cat_15.predict_proba(test)[:,1]

y_out = ((cat_5_out + cat_10_out + cat_15_out)/3 > 0.53).astype('int')
dfdict={}

dfdict["index"]=ind
dfdict["TARGET"]=y_out

df=pd.DataFrame(dfdict)
df.to_csv("solution.csv",index=False,columns=["index","TARGET"])

In [ ]:
'''0.76307 lb current best'''

In [ ]:
'''d 0.75784 lb this stack'''

In [ ]:
'''17630*16 positive pts, xgb 50est 2 depth pos_wt 16 0.76682 val'''

In [ ]:
'''17630 positive points, xgb 100est 3depth pos_wt 10 0.779427868719'''

In [ ]:
'''100-0.768 max depth 2'''

In [78]:
y_out = xgb.predict(test)

dfdict={}

dfdict["index"]=ind
dfdict["TARGET"]=y_out

df=pd.DataFrame(dfdict)
df.to_csv("solution.csv",index=False,columns=["index","TARGET"])

In [24]:
print(X_train.columns)

Index(['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7',
       'col_8', 'col_9',
       ...
       'col_359', 'col_360', 'col_361', 'col_362', 'col_363', 'col_364',
       'col_365', 'col_366', 'col_367', 'col_368'],
      dtype='object', length=369)


In [23]:
print(xgb.feature_importances_)

[ 0.01030928  0.16494845  0.00687285  0.02061856  0.00687285  0.          0.
  0.          0.          0.          0.00687285  0.0137457   0.00687285
  0.02061856  0.00687285  0.          0.          0.02749141  0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0

In [26]:
def get_non_zero_importance(cols,xgb):
    
    non_zero = []
    for i,val in enumerate(xgb.feature_importances_):
        if(val>0.00001):
            non_zero.append(cols[i])
    return non_zero        
    
    

In [27]:
non_zero = get_non_zero_importance(X_train.columns,xgb)

In [31]:
non_zero.append('TARGET')